In [1]:
import pandas as pd
from sqlalchemy import inspect
from sqlalchemy import text

In [2]:
def load_dataframe_to_table(engine,dataframe,table_name ,schema='bronze',if_exists='append'):
    try:
        dataframe.to_sql(name=table_name,
                         con=engine,
                         schema=schema,
                         if_exists=if_exists,
                         index=False,
                         chunksize=1000)
       
    
        with engine.connect() as conn:
            query=text(f"SELECT COUNT(*) FROM {schema}.{table_name}")
            result=conn.execute(query)
            count=result.scalar()
        
        print(f"suceffully loded {count} rows into {schema}.{table_name}")
        return True
    except Exception as e:
        print(f"error while loading data into {schema}.{table_name}:{e}")
        return False

In [3]:
def load_all_tables(engine,dataframes,schema='bronze'):
    
    sucessfully_loaded = []
    
    with engine.connect() as conn:
        conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {schema}"))
    
    for table_name ,df in dataframes.items():
        sucess=load_dataframe_to_table(engine,df,table_name,schema)
        if sucess:
            sucessfully_loaded.append(table_name)
    return sucessfully_loaded
    